# 📝 管理 Questbook 任务

这个 notebook 用于：
- 📊 查看现有任务
- ➕ 添加新任务
- ✏️ 编辑现有任务
- 🗑️ 删除任务
- 🔗 管理任务前置关系

## 使用流程

1. 设置版本代号
2. 选择操作类型
3. 填写任务信息
4. 运行生成代码
5. Commit 到 GitHub

In [5]:
import json
import gzip
import pandas as pd
from pathlib import Path
from datetime import datetime
from IPython.display import display, HTML

print("✓ 导入模块成功")

✓ 导入模块成功


## ⚙️ 配置：选择要操作的版本

In [6]:
# 设置版本代号
VERSION_CODE = "tutorials"  # 修改为你的版本代号

# 路径配置
BASE_DIR = Path("..") / "bin" / "version" / VERSION_CODE
QUEST_JSON_ZH = BASE_DIR / "quest_json.json"
QUEST_JSON_EN = BASE_DIR / "quest_json_en.json"
QUEST_LINE_JSON = BASE_DIR / "quest_line.json"
ATLAS_JSON = BASE_DIR / "quests_icons.json"

# 检查版本是否存在
if not BASE_DIR.exists():
    print(f"❌ 版本目录不存在: {BASE_DIR}")
    print("请先运行 01_init_questbook.ipynb 初始化")
else:
    print(f"✓ 版本目录: {BASE_DIR}")
    
    # 加载现有数据
    with open(QUEST_JSON_ZH, 'r', encoding='utf-8') as f:
        quest_data_zh = json.load(f)
    
    with open(QUEST_JSON_EN, 'r', encoding='utf-8') as f:
        quest_data_en = json.load(f)
    
    with open(QUEST_LINE_JSON, 'r', encoding='utf-8') as f:
        quest_lines = json.load(f)
    
    with open(ATLAS_JSON, 'r', encoding='utf-8') as f:
        atlas_config = json.load(f)
    
    print(f"✓ 加载数据成功")
    print(f"  任务线数: {len(quest_lines)}")
    print(f"  任务总数: {sum(len(v['data']) for v in quest_data_zh.values())}")

✓ 版本目录: ..\bin\version\tutorials
✓ 加载数据成功
  任务线数: 1
  任务总数: 2


## 📊 查看现有任务

In [ ]:
def show_quests():
    """以表格形式显示所有任务"""
    all_quests = []
    
    for questline_name, questline_data in quest_data_zh.items():
        for quest in questline_data['data']:
            # 找前置任务
            prereqs = []
            for link in questline_data['links']:
                if link['target'] == quest['name']:
                    prereqs.append(link['source'])
            
            all_quests.append({
                '任务线': questline_name,
                '任务名': quest['name'],
                '标题': quest['title'],
                '坐标': f"({quest['x']}, {quest['y']})",
                '大小': quest['symbolSize'],
                '类型': '主线' if quest['is_main'] == 1 else '支线',
                '前置': ', '.join(prereqs) if prereqs else '无',
                'ID': quest['quest_id']
            })
    
    df = pd.DataFrame(all_quests)
    return df

# 显示任务列表
df_quests = show_quests()
print(f"\n共有 {len(df_quests)} 个任务：\n")
display(df_quests)

## ➕ 添加新任务

### 1. 配置新任务信息

In [ ]:
# ========== 新任务配置 ==========
NEW_QUEST = {
    # 基本信息
    "questline": "WelcomeQuests",  # 所属任务线（必须已存在）
    "name": "ThirdQuest",          # 任务唯一标识（英文，无空格）
    
    # 显示信息
    "title_zh": "§b§l第三个任务§r",
    "title_en": "§b§lThird Quest§r",
    "desc_zh": "§e这是第三个任务！§r<br/><br/>继续你的冒险。<br/><br/>§6提示§r：可以添加更多任务。",
    "desc_en": "§eThis is the third quest!§r<br/><br/>Continue your adventure.<br/><br/>§6Tip§r: You can add more quests.",
    
    # 位置和样式
    "x": 500,                      # X坐标
    "y": 150,                      # Y坐标
    "size": 50,                    # 图标大小
    "is_main": 1,                  # 1=主线, 0=支线
    
    # 前置任务
    "prereqs": ["GettingStarted"]  # 前置任务的name列表（留空[]表示无前置）
}

print("新任务配置：")
print(f"  任务线: {NEW_QUEST['questline']}")
print(f"  任务名: {NEW_QUEST['name']}")
print(f"  标题: {NEW_QUEST['title_zh']}")
print(f"  类型: {'主线' if NEW_QUEST['is_main'] else '支线'}")
print(f"  前置: {', '.join(NEW_QUEST['prereqs']) if NEW_QUEST['prereqs'] else '无'}")

### 2. 执行添加

In [ ]:
def add_quest(quest_config):
    """添加新任务到数据中"""
    questline_name = quest_config['questline']
    
    # 检查任务线是否存在
    if questline_name not in quest_data_zh:
        print(f"❌ 任务线 '{questline_name}' 不存在！")
        print(f"现有任务线: {', '.join(quest_data_zh.keys())}")
        return False
    
    # 检查任务名是否重复
    existing_names = [q['name'] for q in quest_data_zh[questline_name]['data']]
    if quest_config['name'] in existing_names:
        print(f"❌ 任务名 '{quest_config['name']}' 已存在！")
        return False
    
    # 生成唯一ID
    quest_id = f"{quest_config['name'].lower()}_{int(datetime.now().timestamp())}"
    
    # 创建任务节点（中文）
    node_zh = {
        "name": quest_config['name'],
        "symbolSize": quest_config['size'],
        "symbol": "",
        "x": quest_config['x'],
        "y": quest_config['y'],
        "data": quest_config['desc_zh'],
        "quest_id": quest_id,
        "title": quest_config['title_zh'],
        "is_main": quest_config['is_main'],
        "tooltip": {"show": True}
    }
    
    # 创建任务节点（英文）
    node_en = node_zh.copy()
    node_en['title'] = quest_config['title_en']
    node_en['data'] = quest_config['desc_en']
    
    # 添加到数据
    quest_data_zh[questline_name]['data'].append(node_zh)
    quest_data_en[questline_name]['data'].append(node_en)
    
    # 添加前置关系
    for prereq in quest_config['prereqs']:
        # 检查前置任务是否存在
        if prereq not in existing_names:
            print(f"⚠️ 警告: 前置任务 '{prereq}' 不存在")
            continue
        
        link = {
            "source": prereq,
            "target": quest_config['name'],
            "symbol": ["none", "arrow"],
            "lineStyle": {
                "color": "#00c800" if quest_config['is_main'] else "#666666",
                "width": 4 if quest_config['is_main'] else 2
            },
            "symbolSize": [0, 10]
        }
        quest_data_zh[questline_name]['links'].append(link)
        quest_data_en[questline_name]['links'].append(link)
    
    # 更新图集配置
    icon_key = f"QuestIcon/{questline_name}"
    if icon_key not in atlas_config:
        atlas_config[icon_key] = []
    atlas_config[icon_key].append(quest_id)
    
    # 添加占位符图标
    DUMMY_ICON = "iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAYAAAAfFcSJAAAADUlEQVR42mNk+M9QDwADhgGAWjR9awAAAABJRU5ErkJggg=="
    
    icon_dir = BASE_DIR / "quests_icons" / "QuestIcon" / questline_name
    icon_json_path = icon_dir / f"{questline_name}.json"
    
    if icon_json_path.exists():
        with open(icon_json_path, 'r', encoding='utf-8') as f:
            icon_data = json.load(f)
    else:
        icon_data = {}
    
    icon_data[quest_id] = DUMMY_ICON
    
    with open(icon_json_path, 'w', encoding='utf-8') as f:
        json.dump(icon_data, f, ensure_ascii=False, indent=2)
    
    # 压缩为GTBL
    gtbl_path = icon_json_path.with_suffix('.gtbl')
    with gzip.open(gtbl_path, 'wb') as f:
        f.write(json.dumps(icon_data, ensure_ascii=False).encode('utf-8'))
    
    print(f"✓ 任务添加成功！")
    print(f"  任务名: {quest_config['name']}")
    print(f"  任务ID: {quest_id}")
    print(f"  图标文件: {icon_json_path.relative_to(Path('..'))}")
    
    return True

# 执行添加
if add_quest(NEW_QUEST):
    print("\n📊 更新后的任务列表：")
    display(show_quests())

## ✏️ 编辑现有任务

### 1. 查找要编辑的任务

In [ ]:
# 设置要编辑的任务名
EDIT_QUEST_NAME = "ThirdQuest"  # 修改为要编辑的任务名

# 查找任务
def find_quest(quest_name):
    for questline_name, questline_data in quest_data_zh.items():
        for i, quest in enumerate(questline_data['data']):
            if quest['name'] == quest_name:
                return questline_name, i, quest
    return None, None, None

questline, idx, current_quest = find_quest(EDIT_QUEST_NAME)

if current_quest:
    print(f"找到任务: {EDIT_QUEST_NAME}")
    print(f"  任务线: {questline}")
    print(f"  当前标题: {current_quest['title']}")
    print(f"  当前坐标: ({current_quest['x']}, {current_quest['y']})")
    print(f"  当前大小: {current_quest['symbolSize']}")
else:
    print(f"❌ 未找到任务: {EDIT_QUEST_NAME}")

### 2. 修改任务属性

In [ ]:
# ========== 编辑配置 ==========
# 只修改需要改的字段，不需要修改的保持为 None

EDIT_CONFIG = {
    "title_zh": None,              # 修改中文标题，或保持 None 不修改
    "title_en": None,              # 修改英文标题
    "desc_zh": None,               # 修改中文描述
    "desc_en": None,               # 修改英文描述
    "x": None,                     # 修改X坐标
    "y": None,                     # 修改Y坐标
    "size": None,                  # 修改大小
    "is_main": None,               # 修改类型 (1/0)
}

# 示例：修改坐标和大小
EDIT_CONFIG["x"] = 600
EDIT_CONFIG["y"] = 200
EDIT_CONFIG["size"] = 55

# 执行修改
if current_quest:
    for key, value in EDIT_CONFIG.items():
        if value is not None:
            if key == "title_zh":
                quest_data_zh[questline]['data'][idx]['title'] = value
            elif key == "title_en":
                quest_data_en[questline]['data'][idx]['title'] = value
            elif key == "desc_zh":
                quest_data_zh[questline]['data'][idx]['data'] = value
            elif key == "desc_en":
                quest_data_en[questline]['data'][idx]['data'] = value
            elif key == "x":
                quest_data_zh[questline]['data'][idx]['x'] = value
                quest_data_en[questline]['data'][idx]['x'] = value
            elif key == "y":
                quest_data_zh[questline]['data'][idx]['y'] = value
                quest_data_en[questline]['data'][idx]['y'] = value
            elif key == "size":
                quest_data_zh[questline]['data'][idx]['symbolSize'] = value
                quest_data_en[questline]['data'][idx]['symbolSize'] = value
            elif key == "is_main":
                quest_data_zh[questline]['data'][idx]['is_main'] = value
                quest_data_en[questline]['data'][idx]['is_main'] = value
    
    print(f"✓ 任务 '{EDIT_QUEST_NAME}' 修改成功！")
    print("\n📊 更新后的任务列表：")
    display(show_quests())

## 💾 保存所有更改

In [ ]:
def save_all_data():
    """保存所有数据到文件"""
    # 保存中文任务数据
    with open(QUEST_JSON_ZH, 'w', encoding='utf-8') as f:
        json.dump(quest_data_zh, f, ensure_ascii=False, indent=2)
    print(f"✓ 保存中文数据: {QUEST_JSON_ZH.relative_to(Path('..'))}")
    
    # 保存英文任务数据
    with open(QUEST_JSON_EN, 'w', encoding='utf-8') as f:
        json.dump(quest_data_en, f, ensure_ascii=False, indent=2)
    print(f"✓ 保存英文数据: {QUEST_JSON_EN.relative_to(Path('..'))}")
    
    # 保存图集配置
    with open(ATLAS_JSON, 'w', encoding='utf-8') as f:
        json.dump(atlas_config, f, ensure_ascii=False, indent=2)
    print(f"✓ 保存图集配置: {ATLAS_JSON.relative_to(Path('..'))}")
    
    print("\n✨ 所有更改已保存！")
    print("\n🔧 后续步骤:")
    print("  1. (可选) 使用 03_manage_icons.ipynb 替换图标")
    print("  2. Commit 并 push 到 GitHub")
    print("  3. 等待 GitHub Actions 自动构建")

save_all_data()

## 📈 任务统计

In [ ]:
# 统计信息
total_quests = sum(len(v['data']) for v in quest_data_zh.values())
total_links = sum(len(v['links']) for v in quest_data_zh.values())
main_quests = sum(sum(1 for q in v['data'] if q['is_main'] == 1) for v in quest_data_zh.values())
side_quests = total_quests - main_quests

print("="*50)
print("📊 Questbook 统计")
print("="*50)
print(f"版本: {VERSION_CODE}")
print(f"任务线数: {len(quest_data_zh)}")
print(f"任务总数: {total_quests}")
print(f"  • 主线任务: {main_quests}")
print(f"  • 支线任务: {side_quests}")
print(f"前置关系: {total_links}")
print("="*50)

# 按任务线统计
print("\n📋 各任务线详情:")
for questline_name in quest_data_zh.keys():
    line_quests = len(quest_data_zh[questline_name]['data'])
    line_links = len(quest_data_zh[questline_name]['links'])
    print(f"  {questline_name}: {line_quests} 个任务, {line_links} 个连接")